In [71]:
import time
import heapq
import tracemalloc

#--------------------------------------------------node-------------------------
class Node:
    def __init__(self, state, parent,cost):
        self.state = state
        self.parent = parent
        self.cost = cost

    def __lt__(self, other):
        return self.cost < other.cost

#---------------------------------------------------queue-----------------------
class Queue:
  def __init__(self):
    self.que=[]

  def add(self, Node):
    self.que.append(Node)

  def is_empty(self):
    return True if len(self.que) == 0 else False

  def remove(self):
    if self.is_empty():
      return None
    else:
      node = self.que[0]
      self.que = self.que[1:]
      return node
#-------------------------------------------------------stack-------------------

class Stack:
  def __init__(self):
    self.st=[]
    self.top = -1
  def push(self,Node):
    self.st.append(Node)
    self.top += 1

  def is_empty(self):
    return self.top == -1

  def pop(self):
    if self.is_empty():
        return None
    else:
        self.top -= 1
        return self.st.pop()


#------------------------------------------------------priqueue-----------------
class PriorityQueue:
  def __init__(self):
        self.que = []

  def add(self, node):
        heapq.heappush(self.que, node)

  def remove(self):
        return heapq.heappop(self.que)

  def is_empty(self):
        return len(self.que) == 0


#--------------------------------------------------puzzle-----------------------
class Puzzle_solution:
  def __init__(self,start,goal):
    self.start_state=start
    self.goal_state=goal
    self.found=None
    self.nodes_visited = 0

  def find_neighbours(self,state):
    possible_moves=[]
    row, col =None, None
    for i,rows in enumerate(state):
      for j, val in enumerate(rows):
        if val==0:
          row,col = i,j
          break
    if row > 0:
      pos = [row[:] for row in state]
      pos[row][col] = pos[row - 1][col]
      pos[row - 1][col] = 0
      possible_moves.append(pos)
    if col > 0:
      pos = [row[:] for row in state]
      pos[row][col] = pos[row][col - 1]
      pos[row][col - 1] = 0
      possible_moves.append(pos)
    if row < 2:
      pos = [row[:] for row in state]
      pos[row][col] = pos[row + 1][col]
      pos[row + 1][col] = 0
      possible_moves.append(pos)
    if col < 2:
      pos = [row[:] for row in state]
      pos[row][col] = pos[row][col + 1]
      pos[row][col + 1] = 0
      possible_moves.append(pos)
    return possible_moves


  def print_sol(self,node,path_cost):
    cells = []
    while node.parent is not None:
      path_cost[0] += 1
      cells.append(node.state)
      node = node.parent
    cells.reverse()
    for cell in cells:
      for row in cell:
        for elem in row:
          print(elem, end=' ')
        print()
      print()


#----------------------------------------BFS------------------------------------
  def BFS(self):
    time_taken = 0
    self.nodes_visited = 0
    path_cost = [0]
    start_time = time.time()
    visited = set()
    root = Node(self.start_state, parent=None, cost = 0)
    goal = Node(self.goal_state, parent=None,cost = 1)
    q=Queue()
    q.add(root)
    while True:
      if q.is_empty():
        self.found = False
        return "Not found"
      next_node = q.remove()
      self.nodes_visited += 1
      visited.add(tuple(map(tuple, next_node.state)))
      if (next_node.state == goal.state):
        self.found = True
        end = time.time()
        time_taken = end - start_time
        self.print_sol(next_node,path_cost)
        sol = {"nodes visited":self.nodes_visited,"path cost": path_cost,"time": time_taken}
        return sol
      neighbours = self.find_neighbours(next_node.state)
      for n in neighbours:
       if tuple(map(tuple, n)) not in visited:
           child = Node(state=n, parent=next_node, cost = (next_node.cost)+1 )
           q.add(child)
    pass

#-------------------------------------------DFS---------------------------------


  def DFS(self,depth,simple):
    if simple:
      self.nodes_visited=0
    time_taken = 0
    path_cost = [0]
    start_time = time.time()
    visited = set()
    root = Node(self.start_state, parent=None,cost = 0)
    goal = Node(self.goal_state, parent=None, cost =1)
    s=Stack()
    s.push(root)
    while True:
      if s.is_empty():
        self.found = False
        return None
      next_node = s.pop()
      self.nodes_visited += 1
      visited.add(tuple(map(tuple, next_node.state)))

      if (next_node.state == goal.state):
        self.found = True
        end = time.time()
        time_taken = end - start_time
        self.print_sol(next_node,path_cost)
        sol = {"nodes visited":self.nodes_visited,"path cost": path_cost,"time": time_taken}
        return sol

      if next_node.cost < depth:
        neighbours = self.find_neighbours(next_node.state)
        for n in neighbours[::-1]:
          if tuple(map(tuple, n)) not in visited:
            child = Node(state=n, parent=next_node,cost =(next_node.cost)+1 )
            s.push(child)
    pass

#----------------------------------------------UCS-----------------------------


  def UCS(self):
    time_taken = 0
    self.nodes_visited = 0
    path_cost = [0]
    start_time = time.time()
    visited = []
    root = Node(self.start_state, parent=None,cost=0)
    goal = Node(self.goal_state, parent=None,cost=1)
    q=PriorityQueue()
    q.add(root)
    while True:
      if q.is_empty():
        self.found = False
        return "Not found"
      next_node = q.remove()
      self.nodes_visited += 1
      visited.append(next_node.state)
      if (next_node.state == goal.state):
        self.found = True
        end = time.time()
        time_taken = end - start_time
        self.print_sol(next_node,path_cost)
        path_cost = next_node.cost
        sol = {"nodes visited":self.nodes_visited,"path cost": path_cost, "time": time_taken}
        return sol
      neighbours = self.find_neighbours(next_node.state)
      for state in neighbours:
        if state not in visited:
           costc = (next_node.cost)+1
           child = Node(state=state, parent=next_node,cost=costc )
           q.add(child)
    pass




#-------------------------------------------IDS---------------------------------


  def IDS(self):
    time_taken = 0
    start_time = time.time()
    depth_lim=0

    while True:
      result = self.DFS(depth_lim,False)
      nodes = self.nodes_visited

      if result is not None:
        end_time = time.time()
        time_taken = end_time - start_time
        #self.print_sol(result,path_cost)
        sol = {"nodes visited":nodes,"path cost": result.get("path cost"), "time": time_taken}
        return result

      depth_lim += 1
      print("increasing depth limit",depth_lim)
    pass


#-------------------------------------------Main--------------------------------

node1 = Node(state=[[1, 0, 2], [3, 4, 5], [6, 7, 8]], parent=None,cost =1)
node2 = Node(state=[[3, 1, 2], [0, 4, 5], [6, 7, 8]], parent=None,cost =1)




a= Puzzle_solution(node1.state,node2.state)
print("Start State:")
for row in node1.state:
  for elem in row:
    print(elem, end=' ')
  print()

print("End State:")
for row in node2.state:
  for elem in row:
    print(elem, end=' ')
  print()

print("Algos:",)
print("-------------------BFS--------------------")
tracemalloc.start()
print(a.BFS())
print("memory:",tracemalloc.get_traced_memory()[0])
tracemalloc.stop()
print("-------------------DFS--------------------")
tracemalloc.start()
print(a.DFS(100,True))
print("memory:",tracemalloc.get_traced_memory()[0])
tracemalloc.stop()
print("-------------------UCS--------------------")
tracemalloc.start()
print(a.UCS())
print("memory:",tracemalloc.get_traced_memory()[0])
tracemalloc.stop()
print("-------------------IDS--------------------")
tracemalloc.start()
print(a.IDS())
print("memory:",tracemalloc.get_traced_memory()[0])
tracemalloc.stop()


Start State:
1 0 2 
3 4 5 
6 7 8 
End State:
3 1 2 
0 4 5 
6 7 8 
Algos:
-------------------BFS--------------------
0 1 2 
3 4 5 
6 7 8 

3 1 2 
0 4 5 
6 7 8 

{'nodes visited': 5, 'path cost': [2], 'time': 0.00020694732666015625}
memory: 11324
-------------------DFS--------------------
0 1 2 
3 4 5 
6 7 8 

3 1 2 
0 4 5 
6 7 8 

{'nodes visited': 3, 'path cost': [2], 'time': 0.00010848045349121094}
memory: 13560
-------------------UCS--------------------
0 1 2 
3 4 5 
6 7 8 

3 1 2 
0 4 5 
6 7 8 

{'nodes visited': 8, 'path cost': 2, 'time': 0.00019860267639160156}
memory: 17211
-------------------IDS--------------------
increasing depth limit 1
increasing depth limit 2
0 1 2 
3 4 5 
6 7 8 

3 1 2 
0 4 5 
6 7 8 

{'nodes visited': 16, 'path cost': [2], 'time': 8.726119995117188e-05}
memory: 15754
